In [ ]:
%matplotlib notebook
%matplotlib notebook


import os
from pathlib import Path as P
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime as dt


if 'Demo scripts' in os.getcwd(): os.chdir('..')  # change to main directory
print('Current directory: {}'.format( os.getcwd() ))


#CaImAn stuff
import caiman as cm
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import params as params
from caiman.source_extraction import cnmf
from caiman.utils.visualization import inspect_correlation_pnr, nb_inspect_correlation_pnr


# CASCADE stuff
# import glob
# import scipy.io as sio
# import ruamel.yaml as yaml

# from cascade2p import cascade
# from cascade2p.utils import plot_dFF_traces, plot_noise_level_distribution, plot_noise_matched_ground_truth
# from cascade2p import checks

# checks.check_packages()

# Specify working directories

In [ ]:
path = "New_Analysis_Pipeline_Test/Test_Data/Raw_Data/wfC318_2016_10_18/recording_20161018_155817.hdf5"
# path = input("Enter file path: \n/home/ar4210/engram/anole/Mouse/")
full_data_dir = f"/home/ar4210/engram/anole/Mouse/{path}"

In [ ]:
os.path.exists(full_data_dir)

In [ ]:
path = P(full_data_dir)

home_parts = path.parts[:3]
engram_dir_parts = path.parts[:10]

home = P(*home_parts)
engram_path = P(*engram_dir_parts)
cwd = os.getcwd()

fstem = path.stem # file name without extension
extension = path.suffix # .tif, .hdf5, etc.
fname = path.name # fstem + extension

storage_folder = path.parts[9]

print(home)
print(engram_path)
print(fstem, extension, fname)

In [ ]:
# destination directories
destination_dir     = os.path.join(cwd, "collected_data")
traces_dir          = os.path.join(destination_dir, "Traces")
inferred_spikes_dir = os.path.join(destination_dir, "InferredSpikes")

# CaImAn STEP

In [ ]:
if 'dview' in locals():
    cm.stop_server(dview=dview)
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='local', n_processes=None, single_thread=False)

In [ ]:
f = [f"{full_data_dir}"]

### Motion Correction

#### MoCorr Parameters

In [ ]:
frate = 20                       # movie frame rate
decay_time = 0.4                 # length of a typical transient in seconds

# motion correction parameters
motion_correct = True    # flag for performing motion correction
pw_rigid = True         # flag for performing piecewise-rigid motion correction (otherwise just rigid)
gSig_filt = (3, 3)       # size of high pass spatial filtering, used in 1p data
max_shifts = (5, 5)      # maximum allowed rigid shift
strides = (48, 48)       # start a new patch for pw-rigid motion correction every x pixels
overlaps = (24, 24)      # overlap between pathes (size of patch strides+overlaps)
max_deviation_rigid = 3  # maximum deviation allowed for patch with respect to rigid shifts
border_nan = 'copy'      # replicate values along the boundaries

mc_dict = {
    'fnames': f[0],
    'fr': frate,
    'decay_time': decay_time,
    'pw_rigid': pw_rigid,
    'max_shifts': max_shifts,
    'gSig_filt': gSig_filt,
    'strides': strides,
    'overlaps': overlaps,
    'max_deviation_rigid': max_deviation_rigid,
    'border_nan': border_nan,
    'var_name_hdf5':'images'
}

opts = params.CNMFParams(params_dict=mc_dict)

#### Perform MoCorr

In [ ]:
if motion_correct:
    # do motion correction rigid
    mc = MotionCorrect(f[0], dview=dview, var_name_hdf5 = 'images',**opts.get_group('motion'))
    mc.motion_correct(save_movie=True)
    f_mc = mc.fname_tot_els if pw_rigid else mc.fname_tot_rig
    if pw_rigid:
        bord_px = np.ceil(np.maximum(np.max(np.abs(mc.x_shifts_els)),
                                     np.max(np.abs(mc.y_shifts_els)))).astype(np.int)
    else:
        bord_px = np.ceil(np.max(np.abs(mc.shifts_rig))).astype(np.int)
#         plt.subplot(1, 2, 1); plt.imshow(mc.total_template_rig)  # % plot template
#         plt.subplot(1, 2, 2); plt.plot(mc.shifts_rig)  # % plot rigid shifts
#         plt.legend(['x shifts', 'y shifts'])
#         plt.xlabel('frames')
#         plt.ylabel('pixels')

    bord_px = 0 if border_nan is 'copy' else bord_px
    f_new = cm.save_memmap(f_mc, base_name='memmap_', order='C',
                               border_to_0=bord_px)
else:  # if no motion correction just memory map the file
    f_new = cm.save_memmap(f, base_name='memmap_',
                               order='C', border_to_0=0, dview=dview)

### Load memmap'd file

In [ ]:
Yr, dims, T = cm.load_memmap(f_new)
images = Yr.T.reshape((T,) + dims, order='F')

### CNMF-E

#### Parameters for CNMF-E

In [ ]:
# parameters for source extraction and deconvolution
p = 1               # order of the autoregressive system
K = None            # upper bound on number of components per patch, in general None
gSig = (3, 3)       # gaussian width of a 2D gaussian kernel, which approximates a neuron
gSiz = (13, 13)     # average diameter of a neuron, in general 4*gSig+1
Ain = None          # possibility to seed with predetermined binary masks
merge_thr = .7      # merging threshold, max correlation allowed
rf = 40             # half-size of the patches in pixels. e.g., if rf=40, patches are 80x80
stride_cnmf = 20    # amount of overlap between the patches in pixels
#                     (keep it at least large as gSiz, i.e 4 times the neuron size gSig)
tsub = 2            # downsampling factor in time for initialization,
#                     increase if you have memory problems
ssub = 1            # downsampling factor in space for initialization,
#                     increase if you have memory problems
#                     you can pass them here as boolean vectors
low_rank_background = None  # None leaves background of each patch intact,
#                     True performs global low-rank approximation if gnb>0
gnb = 0             # number of background components (rank) if positive,
#                     else exact ring model with following settings
#                         gnb= 0: Return background as b and W
#                         gnb=-1: Return full rank background B
#                         gnb<-1: Don't return background
nb_patch = 0        # number of background components (rank) per patch if gnb>0,
#                     else it is set automatically
min_corr = .8       # min peak value from correlation image
min_pnr = 10        # min peak to noise ration from PNR image
ssub_B = 2          # additional downsampling factor in space for background
ring_size_factor = 1.4  # radius of ring is gSiz*ring_size_factor


opts = params.CNMFParams(params_dict={'method_init': 'corr_pnr',  # use this for 1 photon
                                'K': K,
                                'gSig': gSig,
                                'gSiz': gSiz,
                                'merge_thr': merge_thr,
                                'p': p,
                                'tsub': tsub,
                                'ssub': ssub,
                                'rf': rf,
                                'stride': stride_cnmf,
                                'only_init': True,    # set it to True to run CNMF-E
                                'nb': gnb,
                                'nb_patch': nb_patch,
                                'method_deconvolution': 'oasis',       # could use 'cvxpy' alternatively
                                'low_rank_background': low_rank_background,
                                'update_background_components': True,  # sometimes setting to False improve the results
                                'min_corr': min_corr,
                                'min_pnr': min_pnr,
                                'normalize_init': False,               # just leave as is
                                'center_psf': True,                    # leave as is for 1 photon
                                'ssub_B': ssub_B,
                                'ring_size_factor': ring_size_factor,
                                'del_duplicates': True,                # whether to remove duplicates from initialization
                                'border_pix': 0})                # number of pixels to not consider in the borders)

In [ ]:
cn_filter, pnr = cm.summary_images.correlation_pnr(images[::5], gSig=gSig[0], swap_dim=False)
nb_inspect_correlation_pnr(cn_filter, pnr)

#### Perform CNMF-E

In [ ]:
cnm = cnmf.CNMF(n_processes=n_processes, dview=dview, Ain=Ain, params=opts)
cnm.fit(images)

#### Evaluate components

In [ ]:
#%% COMPONENT EVALUATION
# the components are evaluated in three ways:
#   a) the shape of each component must be correlated with the data
#   b) a minimum peak SNR is required over the length of a transient
#   c) each shape passes a CNN based classifier

min_SNR = 3            # adaptive way to set threshold on the transient size
r_values_min = 0.85    # threshold on space consistency (if you lower more components
#                        will be accepted, potentially with worst quality)
cnm.params.set('quality', {'min_SNR': min_SNR,
                           'rval_thr': r_values_min,
                           'use_cnn': False})
estimates_object = cnm.estimates.evaluate_components(images, cnm.params, dview=dview)

print(' ***** ')
print('Number of total components: ', len(cnm.estimates.C))
print('Number of accepted components: ', len(cnm.estimates.idx_components))

In [ ]:
cnm.estimates.A

In [ ]:
print(cnm.estimates.A)

#### Refit Data to full FOV

In [ ]:
# cnm2 = cnm.refit(images, dview=dview)

In [ ]:
# estimates_object_2 = cnm2.estimates.evaluate_components(images, cnm2.params, dview=dview)

# print(' ***** ')
# print('Number of total components: ', len(cnm2.estimates.C))
# print('Number of accepted components: ', len(cnm2.estimates.idx_components))

In [ ]:
cm.stop_server(dview=dview)

# CASCADE STEP

### Save Calcium Traces from CaImAn as *.npy file

In [ ]:
np.save(f"Ca_Traces_{fstem}", estimates_object.C[cnm.estimates.idx_components])
# np.save(f"Ca_Traces_{fstem}_2", estimates_object.C[int(len(estimates_object.C / 2)):])


### Define function to load in *.npy file

In [ ]:
def load_neurons_x_time(file_path):
    """Custom method to load data as 2d array with shape (neurons, nr_timepoints)"""
    
    if file_path.endswith('.mat'):
        traces = sio.loadmat(file_path)['dF_traces']

    elif file_path.endswith('.npy'):
        traces = np.load(file_path, allow_pickle=True)
        # if saved data was a dictionary packed into a numpy array (MATLAB style): unpack
        if traces.shape == ():
            traces = traces.item()['dF_traces']
    else:
        raise Exception('This function only supports .mat or .npy files.')
    
    # do here transposing or percent to numeric calculation if necessary
    # traces = traces.T
    # traces = traces / 100
    
    return traces

### Load in calcium traces, plot noise level distributions

In [ ]:
example_file = f'Ca_Traces_{fstem}.npy'

traces = load_neurons_x_time( example_file )
frame_rate = 20


print('Number of neurons in dataset:', traces.shape[0])
print('Number of timepoints in dataset:', traces.shape[1])

# interactive plotting
# %matplotlib notebook

noise_levels = plot_noise_level_distribution(traces,frame_rate)

# plt.rcParams['figure.figsize'] = (8.0, 8.0)
# np.random.seed(3452)
# neuron_indices = np.random.randint(traces.shape[0], size=10)
# plot_dFF_traces(traces,neuron_indices,frame_rate)

### List available models from CASCADE

In [ ]:
cascade.download_model( 'update_models',verbose = 1)

yaml_file = open('Pretrained_models/available_models.yaml')
X = yaml.load(yaml_file, Loader=yaml.Loader)
list_of_models = list(X.keys())
print('\n List of available models: \n')
for model in list_of_models:
    print(model)

### Pick a model from above and run it on your data.

In [ ]:
model_name = 'Global_EXC_20Hz_smoothing200ms_causalkernel'

cascade.download_model( model_name,verbose = 1)
spike_rates = cascade.predict( model_name, traces )

### Create DataFrames for Ca2+ traces, CaImAn inferred spikes, CASCADE 100ms smoothing inferred spikes, and CASCADE 200ms inferred spikes.

In [ ]:
ca_traces_df = pd.DataFrame(np.array(cnm.estimates.C[cnm.estimates.idx_components]))
caiman_s_df = pd.DataFrame(np.array(cnm.estimates.S[cnm.estimates.idx_components]))
# cascade_s_df = pd.DataFrame(spike_rates)#[cnm.estimates.idx_components])

### Preprocessing of data

CaImAn ca_traces are on a much greater scale than CaImAn inferred spikes, so ca_traces_df can be divided by 100,000.<br>
CASCADE DataFrames are filled with NaN values so they are replaced with zeroes.

In [ ]:
ca_traces_df = ca_traces_df / 1000000
# cascade_s_df = cascade_s_df.fillna(value = 0)

### Plot Ca2+ traces and compare CaImAn and CASCADE 100ms smoothing

In [ ]:
# %matplotlib inline
import random

print(f"There are {len(cnm.estimates.C)} total neurons. {len(cnm.estimates.idx_components)} of them are acceptable, while {len(cnm.estimates.idx_components_bad)} were rejected.")
neuron = random.randrange(len(cnm.estimates.idx_components))#int(input(f"Pick a neuron from 0 to {len(cnm.estimates.idx_components)-1}\n\n****\n\n"))
  

fig, axs = plt.subplots(2 , 1, figsize = (20,7.5))
x = ca_traces_df.columns

fig.suptitle(f"Neuron {neuron} out of {len(cnm.estimates.idx_components)}")

axs[0].plot(ca_traces_df.loc[neuron], color='green')
axs[0].set_title("Calcium Traces from CaImAn")
# axs[0 , 1].plot(ca_traces_df.loc[neuron], color='green')
# axs[0 , 1].set_title("Calcium Traces from CaImAn")


axs[1].plot(caiman_s_df.loc[neuron], color = 'black')
axs[1].set_title("Inferred Spikes (CaImAn)")
# axs[1 , 1].plot(cascade_s_df.loc[neuron], color = 'black')
# axs[1 , 1].set_title("Inferred Spikes (CASCADE, causal 200ms smoothing)")

# axs[1 , 0].sharey(axs[1 , 1])

plt.show()

In [ ]:
now = dt.now()

ctime = now.strftime("%Y%m%d_%H%M%S")

if not os.path.exists(f"{destination_dir}/{storage_folder}"):
    os.mkdir(f"{destination_dir}/{storage_folder}")
    
plt.savefig(f"{destination_dir}/{storage_folder}/{fstem}_CA1_PLOT_causal_200ms_{ctime}.png")
# cascade_s_df.to_csv(f"{destination_dir}/{storage_folder}/{fstem}_CascadeSpikes_{ctime}.csv")
# ca_traces_df.to_csv(f"{destination_dir}/{storage_folder}/{fstem}_Traces_{ctime}.csv")
    